In [1]:
import json
import regex as re

In [11]:
cwd = os.getcwd()
cwd

'/Users/justinchen/Desktop/Synthetic Data'

In [16]:
def get_coords(file):
    final_list = []
    if file['annotations'] == []:
        return final_list
    for i in range(len(file['annotations'])):
        temp_list = []
        temp_list.append(0) #file['annotations'][i]['id']
        for j in range(len(file['annotations'][i]['bbox'])):
            temp_list.append(file['annotations'][i]['bbox'][j])
        final_list.append(temp_list)
    final_list.append(file['filename'])
    return final_list

In [17]:
def calc_new_coords(coord_list, img_size):
    new_coord_list = []
    temp_coord_list = coord_list[:-1]
    for box in temp_coord_list:
        new_box = [box[0]]
        x_min = box[1]
        y_min = box[2]
        w = box[3]
        h = box[4]
        
        x_max = x_min + w
        y_max = y_min + h
        
        x_cent = (x_min + x_max) / 2
        y_cent = (y_min + y_max) / 2
        print(x_cent)
        
        new_box.extend([x_cent / img_size, y_cent / img_size, w / img_size, h / img_size])
        new_coord_list.append(new_box)
    new_coord_list.append(coord_list[-1])
    return new_coord_list

In [22]:
def write_txt(yolo_list):
    if yolo_list == []:
        return 
    file_name = yolo_list[-1][:-4]
    #new directory to save txt file to
    save_path = '/Users/justinchen/Desktop/Synthetic Data/0.05Dataset/obj'
    completeName = os.path.join(save_path, file_name)
    f = open(completeName + ".txt", "x")
    for i in range(len(yolo_list)-1):
        for j in range(len(yolo_list[i])):
            f.write(str(yolo_list[i][j]) + " ")
        f.write("\n")
    print('txt file written')
    f.close()

In [102]:
import os

In [ ]:
import os
# assign directory
directory = '/Users/justinchen/Desktop/Synthetic Data/2fa4cf7b-b812-491a-a9c6-e6bc1bdc7f15/annotations'
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        JsonFile = open(f)
        file = json.load(JsonFile)
        coord_list = get_coords(file)
        if not coord_list == []:
            new_coord_list = calc_new_coords(coord_list, 320)
            print(new_coord_list)
            write_txt(new_coord_list)

In [24]:
directory = '/Users/justinchen/Desktop/Synthetic Data/0.05Dataset/obj'
count = 0
for filename in os.listdir(directory):
    newfilename = filename.split('.')[0]
#     print(filename)
#     print(filename + ".txt")
    if not os.path.exists(directory + '/' + newfilename + ".txt"):
        print(filename)
        os.remove(directory + '/' + filename)
        count += 1
print("Images removed:", count)
print("Images left:", 1500-count)

0000001778-1-Image.jpeg
0000000280-1-Image.jpeg
0000000308-1-Image.jpeg
0000001179-1-Image.jpeg
0000001417-1-Image.jpeg
0000001333-1-Image.jpeg
0000000661-1-Image.jpeg
0000000634-1-Image.jpeg
0000000098-1-Image.jpeg
0000001193-1-Image.jpeg
0000000493-1-Image.jpeg
0000000832-1-Image.jpeg
0000000295-1-Image.jpeg
0000001979-1-Image.jpeg
0000000027-1-Image.jpeg
0000000704-1-Image.jpeg
0000001809-1-Image.jpeg
0000000157-1-Image.jpeg
0000001681-1-Image.jpeg
0000000578-1-Image.jpeg
0000001078-1-Image.jpeg
0000001087-1-Image.jpeg
0000001709-1-Image.jpeg
0000001948-1-Image.jpeg
0000001516-1-Image.jpeg
0000001544-1-Image.jpeg
0000000642-1-Image.jpeg
0000001038-1-Image.jpeg
0000000811-1-Image.jpeg
0000000934-1-Image.jpeg
0000000843-1-Image.jpeg
0000000816-1-Image.jpeg
0000000173-1-Image.jpeg
0000001275-1-Image.jpeg
0000001220-1-Image.jpeg
0000000720-1-Image.jpeg
0000001350-1-Image.jpeg
0000000174-1-Image.jpeg
0000001908-1-Image.jpeg
0000001227-1-Image.jpeg
0000001794-1-Image.jpeg
0000000833-1-Ima

In [14]:
def write_data():
    f = open("obj.data", "x")
    f.write('classes = 1')
    f.write("\n")
    
    f.write('train = data/train.txt')
    f.write("\n")
    
    f.write('valid = data/test.txt')
    f.write("\n")
    
    f.write('names = data/obj.names')
    f.write("\n")
    
    f.write('backup = /mydrive/yolov4/training')
    f.write("\n")
    
    print('.data file written')
    f.close()

In [15]:
write_data()

.data file written


In [95]:
def write_names():
    f = open("obj.names", "x")
    f.write('polar bear')
    f.close()

In [97]:
write_names()